In [3]:
import os, sys
ggmclassifier_path = os.path.abspath(os.path.join(os.getcwd(), '../../GGMClassifier'))
sys.path.insert(0, ggmclassifier_path)
import ggmclassifier as ggmc

import pandas as pd

!pwd

/Users/tomrix/Library/CloudStorage/OneDrive-ImperialCollegeLondon/projects/untargeted_sspa/notebooks


In [15]:
feature_df = pd.read_csv('../data/trans_omic_untargeted/prs_feat_table_pos.csv')
feature_df = feature_df.loc[:, ~feature_df.columns.isin(['namecustom', 'npeaks', '.', 'peakidx', 'ms_level'])]
feature_df.insert(0, 'peak_id', range(1, len(feature_df)+1))
feature_df

,peak_id,mz,mzmin,mzmax,rt,rtmin,rtmax,20P02820001_BEHC18_POS,20P02820002_BEHC18_POS,20P02820006_BEHC18_POS,...,20P02820306_BEHC18_POS,20P02820307_BEHC18_POS,20P02820310_BEHC18_POS,20P02820315_BEHC18_POS,20P02820317_BEHC18_POS,20P02820320_BEHC18_POS,20P02820322_BEHC18_POS,20P02820323_BEHC18_POS,20P02820326_BEHC18_POS,20P02820328_BEHC18_POS
0,1,70.065243,70.064623,70.065658,44.397226,24.481733,62.890923,9.586599e+06,7.135026e+06,6.042677e+06,...,1.357908e+07,1.404549e+07,1.346850e+07,1.681890e+07,1.155169e+07,1.347282e+07,1.119282e+07,9.292339e+06,1.043737e+07,1.438223e+07
1,2,70.162528,70.158316,70.162604,573.480957,572.720947,591.545166,1.527338e+05,1.254979e+05,9.983485e+04,...,1.251664e+05,8.001308e+04,1.651304e+05,1.372039e+05,2.267763e+05,1.766325e+05,6.652003e+04,1.236434e+05,6.293933e+04,1.506385e+05
2,3,70.568609,70.568346,70.568643,582.715912,582.141541,598.514526,1.630222e+06,1.679354e+06,1.773759e+06,...,1.526437e+06,1.618204e+06,1.390878e+06,1.533255e+06,1.486871e+06,1.558230e+06,1.775366e+06,1.523314e+06,1.462615e+06,1.593489e+06
3,4,70.718387,70.711484,70.718776,490.996552,490.394043,499.445557,7.690562e+05,8.072315e+05,6.793675e+05,...,3.965214e+05,8.420217e+05,7.019939e+05,6.955714e+05,5.382372e+05,5.283802e+05,5.148796e+05,6.100374e+05,4.969427e+05,6.217621e+05
4,5,70.819398,70.814585,70.823641,582.642090,576.409241,599.342285,2.278647e+05,2.587944e+05,1.793223e+05,...,1.825304e+05,1.940488e+05,1.607462e+05,1.665075e+05,2.195927e+05,1.966584e+05,1.830991e+05,2.031128e+05,1.695326e+05,1.942003e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4054,4055,1013.654106,1013.653046,1013.658103,569.336853,553.465698,569.549805,3.387082e+06,2.915657e+06,2.035393e+06,...,2.156486e+06,2.990340e+06,3.095060e+06,3.996117e+06,3.090744e+06,3.109460e+06,1.833830e+06,2.830819e+06,1.640916e+06,2.845244e+06
4055,4056,1014.658321,1014.656706,1014.660826,569.448608,552.943604,570.019470,1.837005e+06,1.613000e+06,1.166852e+06,...,1.357932e+06,1.633428e+06,1.710665e+06,1.861301e+06,1.721583e+06,1.543918e+06,9.666535e+05,1.673647e+06,8.348217e+05,1.477050e+06
4056,4057,1020.806625,1020.804925,1020.809015,37.102095,36.475246,37.949020,2.004014e+05,1.515335e+05,2.663923e+05,...,2.298926e+05,2.517092e+05,2.181655e+05,1.954623e+05,1.527324e+05,1.469564e+05,2.203481e+05,2.323592e+05,1.810723e+05,2.279131e+05
4057,4058,1042.802092,1042.800393,1042.805076,37.653584,36.985909,38.529873,2.039884e+06,8.311646e+05,1.344478e+06,...,8.118902e+05,9.839881e+05,5.694706e+05,8.936563e+05,7.257604e+05,5.981037e+05,5.509662e+05,8.117037e+05,8.741196e+05,9.174076e+05


In [ ]:
feat_dicts = ggmc.FeatDicts(feature_df)
ggm_input, feat_labels = ggmc.ggm_classify.prs_npyc_to_ggm(feature_df)
ggm_mx = ggmc.ggm_classify.EstGGM(ggm_input, feat_labels).estimate_ggm()
ggm_mx.to_csv('../data/pos_ggm_mx.csv')
ggm = ggmc.ggm_classify.GGM('../data/pos_ggm_mx.csv')  

Estimating optimal shrinkage intensity lambda (correlation matrix): 0.2177 

Estimate (local) false discovery rates (partial correlations):
Step 1... determine cutoff point
Step 2... estimate parameters of null distribution and eta0
Step 3... compute p-values and estimate empirical PDF/CDF
Step 4... compute q-values and local fdr


Significant edges:  497101 
    Corresponding to  6.04 %  of possible edges 


In [80]:
edge_df = ggmc.mz_picker.get_edge_df(ggm, feat_dicts)
edge_df = ggmc.mz_picker.filter_edge_df(edge_df, rt_delta=1, only_pos_pcor=False)
peak_picker = ggmc.mz_picker.PickPeaks(edge_df, bw=0.0005)
edges_clf = ggmc.mz_picker.classify_edges(peak_picker.get_peak_df(), edge_df)

peak_picker.get_peak_df(n_thresh=20).sort_values(by='n_edges', ascending=False).head(20)

,mz,n_edges,width,mzmin,mzmax,sigma,density,mz_peak_id
9,1.003349,405,0.001446,1.002596,1.004041,0.000294,9.098920,9
5,0.501628,236,0.001217,0.501012,0.502229,0.000255,5.468345,5
13,1.997112,183,0.002978,1.995053,1.998031,0.000390,2.324086,13
43,18.010592,154,0.001239,18.009978,18.011217,0.000265,3.579207,43
35,14.015659,124,0.001329,14.014988,14.016316,0.000279,2.881103,35
74,67.987391,98,0.001775,67.986639,67.988414,0.000435,1.691634,74
57,32.026212,85,0.001126,32.025736,32.026862,0.000276,1.982952,57
54,27.994923,74,0.001287,27.994324,27.995611,0.000292,1.675680,54
84,135.975446,66,0.002413,135.973748,135.976161,0.000304,0.792665,84
47,21.982075,65,0.001184,21.981435,21.982619,0.000231,1.508660,47


In [71]:
esi_df = pd.read_csv('/Users/tomrix/Library/CloudStorage/OneDrive-ImperialCollegeLondon/projects/classnet/mz_picking/esi_mass_differences.csv')
esi_df

,Rank,Density,m/z difference (experimental),m/z difference (theoretical),Annotation,Charge,Annotation class,Density.1,Unnamed: 8
0,1,48.07,1.0033,1.0034,Carbon (12C and 13C; M+1),1,Isotope,48.07,NaN
1,2,17.26,67.9874,67.9874,CHO2Na (sodium formate),1,Neutral adduct,17.26,NaN
2,3,16.85,1.9971,1.9971,Chlorine (35Cl and 37Cl),1,Isotope,16.85,NaN
3,4,15.56,0.5016,0.5017,13C - 12C difference,2,Multiple charge state,15.56,NaN
4,5,13.89,21.9819,21.9819,[M+H]+ - [M+Na]+ difference,1,Charged adduct,13.89,NaN
...,...,...,...,...,...,...,...,...,...
266,260,0.10,19.9682,Not annotated,Not annotated,Not annotated,Not annotated,0.10,NaN
267,260,0.10,0.6657,0.6657,37Cl - 35Cl difference,3,Multiple charge state,0.10,NaN
268,260,0.10,70.0782,70.0783,C5H10,1,Other,0.10,NaN
269,260,0.10,18.0469,Not annotated,Not annotated,Not annotated,Not annotated,0.10,NaN


In [81]:
ggmc.mz_picker.identify_mz_diffs(peak_picker.get_peak_df(n_thresh=20), esi_df).head(30)

,Annotation,Annotation class,m/z difference (theoretical),m/z difference (experimental),n_edges,dunn_rank,mz_peak_id,mz_centre,mzmin,mzmax,width
0,Carbon (12C and 13C; M+1),Isotope,1.0034,1.0033,405.0,1,9.0,1.003349,1.002596,1.004041,0.001446
1,13C - 12C difference,Multiple charge state,0.5017,0.5016,236.0,4,5.0,0.501628,0.501012,0.502229,0.001217
2,Iron (54Fe and 56Fe) or Sulphur (32S and 34S),Isotope,1.9958 (32-34S diff) OR 1.9953 (Fe isotopes),1.9955,183.0,28,13.0,1.997112,1.995053,1.998031,0.002978
3,Chlorine (35Cl and 37Cl),Isotope,1.9971,1.9971,183.0,3,13.0,1.997112,1.995053,1.998031,0.002978
4,H20 (water),Neutral adduct and in-source fragment and biol...,18.0106,18.0106,154.0,7,43.0,18.010592,18.009978,18.011217,0.001239
5,CH2,Neutral adduct and in-source fragment and biol...,14.0157,14.0156,124.0,11,35.0,14.015659,14.014988,14.016316,0.001329
6,CHO2Na (sodium formate),Neutral adduct,67.9874,67.9874,98.0,2,74.0,67.987391,67.986639,67.988414,0.001775
7,CH4O (methanol),Neutral adduct,32.0262,32.0263,85.0,79,57.0,32.026212,32.025736,32.026862,0.001126
8,CO,In-source fragment and biological transformation,27.9949,27.9948,74.0,32,54.0,27.994923,27.994324,27.995611,0.001287
9,CH3N3OPS or C5N2OS,Other,135.9735 or 135.9732,135.9738,66.0,107,84.0,135.975446,135.973748,135.976161,0.002413
